In [74]:
import numpy as np
import pandas as pd


pd_data = pd.read_csv('전국평생학습강좌표준데이터.csv', encoding='cp949')
# print(pd_data.shape)
# print(pd_data.dtypes)
# print(pd_data.head(3))
for e in enumerate(pd_data.columns):
    print(e)
dorp_list = ['교육시작시각','교육종료시각','강좌내용','운영기관전화번호']
pd_data.drop(dorp_list, axis=1, inplace=True)
print(pd_data.dtypes)

(0, '강좌명')
(1, '강사명')
(2, '교육시작일자')
(3, '교육종료일자')
(4, '교육시작시각')
(5, '교육종료시각')
(6, '강좌내용')
(7, '교육대상구분')
(8, '교육방법구분')
(9, '운영요일')
(10, '교육장소')
(11, '강좌정원수')
(12, '수강료')
(13, '교육장도로명주소')
(14, '운영기관명')
(15, '운영기관전화번호')
(16, '접수시작일자')
(17, '접수종료일자')
(18, '접수방법구분')
(19, '선정방법구분')
(20, '홈페이지주소')
(21, '직업능력개발훈련비지원강좌여부')
(22, '학점은행제평가(학점)인정여부')
(23, '평생학습계좌제평가인정여부')
(24, '데이터기준일자')
(25, '제공기관코드')
(26, '제공기관명')
강좌명                object
강사명                object
교육시작일자             object
교육종료일자             object
교육대상구분             object
교육방법구분             object
운영요일               object
교육장소               object
강좌정원수               int64
수강료                 int64
교육장도로명주소           object
운영기관명              object
접수시작일자             object
접수종료일자             object
접수방법구분             object
선정방법구분             object
홈페이지주소             object
직업능력개발훈련비지원강좌여부    object
학점은행제평가(학점)인정여부    object
평생학습계좌제평가인정여부      object
데이터기준일자            object
제공기관코드             object
제공기관명              ob

In [75]:
for col_name in pd_data.columns:
    if '일자' in col_name:
        pd_data[col_name] = pd_data[col_name].astype(np.datetime64, copy=False)
    elif ('구분' in col_name) or ('여부' in col_name) or ('코드' in col_name) or ('요일' in col_name):
        pd_data[col_name] = pd_data[col_name].astype('category', copy=False)
    
print(pd_data.dtypes)

강좌명                        object
강사명                        object
교육시작일자             datetime64[ns]
교육종료일자             datetime64[ns]
교육대상구분                   category
교육방법구분                   category
운영요일                     category
교육장소                       object
강좌정원수                       int64
수강료                         int64
교육장도로명주소                   object
운영기관명                      object
접수시작일자             datetime64[ns]
접수종료일자             datetime64[ns]
접수방법구분                   category
선정방법구분                   category
홈페이지주소                     object
직업능력개발훈련비지원강좌여부          category
학점은행제평가(학점)인정여부          category
평생학습계좌제평가인정여부            category
데이터기준일자            datetime64[ns]
제공기관코드                   category
제공기관명                      object
dtype: object


In [76]:
# - 항목(column)별 결측치/오류치/이상피 갯수 파악

# pd_data.hasnans 
# pd_data['제공기관명'].hasnans
# pd_data['제공기관명'].isna().value_counts()
# 결측치가 있는지 없는지
t = []
for col_name in pd_data.columns:
   
    if pd_data[col_name].hasnans == True:
#         print(col_name)
        cnt_s = pd_data[col_name].isna().value_counts()
        cnt = cnt_s.to_numpy()
        t.append([col_name, cnt[0], cnt[1]])
# print(t)
t= np.array(t)
missing_df = pd.DataFrame(t[:,1:],index = t[:,0], columns=['value_count', 'NaN_count'])
# missing_df.index = missing_df['col_name']
print(missing_df)

       value_count NaN_count
운영요일         22007      4528
교육장소         23755      2780
접수시작일자       21516      5019
접수종료일자       21474      5061
선정방법구분       22273      4262
홈페이지주소       24221      2314


## 정제 요건 정의

* 운영요일의 결측치를 어떻게 할 것인가?
    - 운영요일별 카운트시 운영요일을 특정할 수 없으므로 삭제 조치를 하도록함.
* 교육장소의 결측치를 어떻게 할 것인가?
    - 만약, '홈페이지주소' 가 있을 경우, '온라인'으로 결측지를 대체, 홈페이지 주속가 없을 경우 삭제 조치
    - '교육방법 구분'이 '온라인'으로 되어있을 경우, '온라인'으로 결측치를 대체 그렇지 않을 경우, 삭제
* 접수시작일자, 접수종료일자 결측치를 어떨게 할 것인가?
    - '교육방법 구분'이 '온라인'으로 되어있을 경우, '온라인'으로 결측치를 대체 그렇지 않을 경우, 삭제
* 선정방법구분의 결측치를 어떻게 할 것인가?
    - 선정방법구분 별 카운트시 선정방법구분을 특정할 수 없으므로 삭제 조치를 하도록함.

In [77]:
# f1 = pd_data['운영요일'].isna()
# pd_data['운영요일'][f1]

pd_data['운영요일'].value_counts()

# 운영요일별 카운트시 운영요일을 특정할 수 없으므로
# 삭제 조치를 하도록함.

목            2556
화            2524
수            2434
월            1937
금            1851
             ... 
월, 목            1
월+화+토           1
월+화+수+금+토       1
월+화+목+토         1
화요일~금요일         1
Name: 운영요일, Length: 218, dtype: int64

In [78]:
# pd_data['교육장소'].value_counts()
#'교육방법 구분'이 '온라인'으로 되어있을 경우, '온라인'으로 결측치를 대체 
# 그렇지 않을 경우, 삭제
pd_data['교육방법구분'].value_counts()
filter1 = pd_data['교육장소'].isna()

# print(pd_data['교육방법구분'][filter1].value_counts())

filter2 = pd_data['교육방법구분']=='온라인'
pd_data['교육장소'][filter2]='온라인'

pd_data['교육장소'].isna().value_counts()

C:\Users\ITPS\AppData\Local\Temp\ipykernel_11804\1640146004.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['교육장소'][filter2]='온라인'


False    24706
True      1829
Name: 교육장소, dtype: int64

In [79]:
# 범용으로 사용
for i in range(pd_data.shape[0]):
    row = pd_data.iloc[i, :]
    if (row['교육장소']==np.NaN) and (row['교육방법구분']=='온라인'):
        row['교육장소'] = '온라인'


In [70]:
pd_data['교육장소'].value_counts()

# 만약, '홈페이지주소' 가 있을 경우, '온라인'으로 결측지를 대체
# 홈페이지 주속가 없을 경우 삭제 조치

#'교육방법 구분'이 '온라인'으로 되어있을 경우, '온라인'으로 결측치를 대체 
# 그렇지 않을 경우, 삭제

f1 = pd_data['교육장소'].isna()
f2 = pd_data['교육방법구분'].value_counts()
# print(f2)
# sub_data = pd_data[['교육방법구분','교육장소']]
# g1 = sub_data.groupby('교육방법구분')
pd_data_f = pd_data[f1].apply(lambda x: x)

lista = list(pd_data_f.columns)

for e in pd_data_f.colums:
    if '온라인' in pd_data_f['교육방법구분']:
        pd_data_f['교육방법구분'] = pd_data_f['교육방법구분'].replcace(np.NaN, '온라인')
    else:
        


온라인                        4453
광명시여성비전센터                   572
평생학습관                       562
광산구청소년수련관                   525
아트센터달                       224
                           ... 
연희동 아시아나 골프죤 or 연희동자치회관       1
대신야학                          1
북아현동주민센터 3층                   1
홍은종합사회복지관 별관1층                1
화성센트럴파크 도서관                   1
Name: 교육장소, Length: 3163, dtype: int64

In [46]:
pd_data['선정방법구분'].value_counts()

선착순         16861
추첨           1219
제한없음          516
추첨+선착순        451
인터넷 추첨        392
            ...  
방문접수후 면접        1
방문심사            1
읍면 추천           1
군부대별 선정         1
자체심사            1
Name: 선정방법구분, Length: 72, dtype: int64